In [2]:
import pandas as pd
import numpy as np

The Simple Recommender

In [3]:
df = pd.read_csv('\computer vision\datasets\movies_metadata.csv.xls')

C:\Users\mchan\AppData\Local\Temp\ipykernel_12008\3467934803.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('\computer vision\datasets\movies_metadata.csv.xls')


In [4]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [5]:
df.shape

(45466, 24)

In [6]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

we want to consider
   -choosing a metric
   - decide on the prerequisites for the movie to be the featured on the chart
   - calculate the sore for every movie that satisfies the conditions
-  - output the list of the movies in decreasing order of their scores 

The Metric

- the metric is the numeric quantity on which we rank the movies

rating : lets say a movie with 8/10 with viewers 10000 and a movie with 9.5/10 with viewers 100 cannot be taken into considerations just by rating

we need a metric which can, to an extend take into account both the movie rating and the no of votes(popularity)

-this would give a greater preference to a blockbuster movie rated 8 by 10000 user over an art movie rated 9 by 100 users.

-movie with few ratings are not much reliable
eg - A movie rated 10/10 by 5 users does not necessarily means that it is agood movie.

-here we are using imdb's weighted rating formula

THE PREREQUISITES

-There is no fixed value for m(it is an arbitrary value)
-For our purpose we are using 80th precentile movie as our value for m.
-For a movie to be considered in the rankings it must have garnered more votes than atleast 80% of the movies present in our dataset

In [21]:
mal_movie = df[df['original_language'] == 'ml']

In [8]:
#calculate the no of votes garnered by the 80th percentile movie
m = mal_movie['vote_count'].quantile()
m

6.0

-Consider movie only more than 45 minutes and less than 300 minutes

In [9]:
#movies >45 and <300
q_movies = mal_movie[(mal_movie['runtime'] >= 45) & (mal_movies['runtime'] < 300)]
q_movies = q_movies[q_movies['vote_count'] >= m]

C:\Users\mchan\AppData\Local\Temp\ipykernel_12008\2663027419.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q_movies = mal_movie[(mal_movie['runtime'] >= 45) & (df['runtime'] < 300)]


In [10]:
mal_movie.shape

(36, 24)

In [11]:
q_movies.shape

(21, 24)

CACULATE THE SCORE

In [12]:
c = mal_movie['vote_average'].mean()

In [13]:
c

6.383333333333333

In [14]:
#function to calculate the imdb weighted rating for each movie
def weighted_rating(x,m=m,c=c):
    v = x['vote_count']
    R = x['vote_average']
    #compute the weighted rating
    return (v/(v+m)*R) + (m/(m+v) * c)

In [15]:
q_movies['score'] = q_movies.apply(weighted_rating,axis=1)

In [16]:
q_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
22148,False,NaN,110000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 10402, ...",NaN,180048,tt2786278,ml,ആമേൻ,"Solomon, who belongs to a band, is in love wit...",...,270000.0,165.0,"[{'iso_639_1': 'ml', 'name': ''}]",Released,What we play is life,Amen,False,6.9,6.0,6.641667
25761,False,NaN,750000,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",http://aashirvadcinemas.com/view_details.php?m...,244049,tt3417422,ml,ദൃശ്യം,Georgekutty lives a happy life with his wife a...,...,12000000.0,160.0,"[{'iso_639_1': 'ml', 'name': ''}]",Released,Visuals can be deceiving,Drishyam,False,8.1,23.0,7.744828
32016,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,207649,tt0202055,ml,വാനപ്രസ്ഥം,"Kunhikuttan, a famous Kathakali dancer, meets ...",...,0.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Vanaprastham,False,7.2,6.0,6.791667
32206,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",https://www.facebook.com/MemoriesMalayalamMovie,217389,tt3124456,ml,മെമ്മറീസ്,"Prithviraj is a police officer named Sam Alex,...",...,0.0,132.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An Episode in the Life of a Police Officer...!,Memories,False,7.2,9.0,6.873333
32207,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",NaN,190945,tt2924472,ml,മുംബൈ പോലീസ്,Mumbai Police tells the friendship of 3 police...,...,0.0,145.0,"[{'iso_639_1': 'ml', 'name': ''}, {'iso_639_1'...",Released,NaN,Mumbai Police,False,7.4,10.0,7.018750
33163,False,NaN,7400000,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,268660,tt3668162,ml,ബാംഗ്ലൂർ ഡെയ്സ്,A fun roller coaster ride about three young pe...,...,1300000.0,172.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Three cousins set out to fulfill their Bangalo...,Bangalore Days,False,7.4,19.0,7.156000
33164,False,NaN,220000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,257160,tt3517192,ml,ഓം ശാന്തി ഓശാന,"Pooja Mathew (Nazriya Nazim), daughter of Doct...",...,1500000.0,130.0,"[{'iso_639_1': 'ml', 'name': ''}]",Released,NaN,Ohm Shanthi Oshaana,False,6.9,8.0,6.678571
34216,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,278393,tt4176604,ml,വർഷം,"The film narrates the tale of Venu, owner of a...",...,0.0,140.0,"[{'iso_639_1': 'ml', 'name': ''}]",Released,NaN,Varsham,False,6.7,6.0,6.541667
34640,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 27, 'na...",NaN,84368,tt0214915,ml,മണിച്ചിത്രത്താഴ്,"A young couple, Ganga (Shobana) and Nakulan (S...",...,0.0,169.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}, {'iso_6...",Released,The Ornate Lock,Manichitrathazhu,False,7.4,9.0,6.993333
34641,False,"{'id': 415699, 'name': 'Devasuram Collection',...",0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,237672,tt0291855,ml,ദേവാസുരം,"Neelakantan, heir to his father's fortunes, is...",...,0.0,160.0,"[{'iso_639_1': 'ml', 'name': ''}]",Released,NaN,Devasuram,False,6.4,8.0,6.392857


SORTING AND OUTPUT

In [17]:
q_movies.sort_values(by='score',ascending=False,inplace = True)

In [18]:
q_movies[['title','vote_count','vote_average','score','runtime']]

,title,vote_count,vote_average,score,runtime
25761,Drishyam,23.0,8.1,7.744828,160.0
41520,Kammatipaadam,9.0,8.1,7.413333,177.0
40323,Maheshinte Prathikaaram,9.0,7.7,7.173333,121.0
33163,Bangalore Days,19.0,7.4,7.156000,172.0
41521,Action Hero Biju,6.0,7.7,7.041667,144.0
35270,Shutter,6.0,7.7,7.041667,134.0
39501,Charlie,10.0,7.4,7.018750,130.0
32207,Mumbai Police,10.0,7.4,7.018750,145.0
34640,Manichitrathazhu,9.0,7.4,6.993333,169.0
44285,Ustad Hotel,15.0,7.2,6.966667,150.0


malayalam top movie recommendation